### ToDo
- Train Random Forest as a baseline for each assay in the validation and test splits. For each assay, a support set of ten data points (5 active and 5 inactive molecules) will be created, on which the Random Forest will be trained, and a query set will be used to evaluate the predictions.
- Train Neural Network as a Frequent Model: All training assays will be aggregated into a pool. The neural network (Feedforward Neural Network) will be trained based on the aggregated data to achieve the best possible performance on the validation set.
- Compare the performance of the models using AUC. For Random Forest, calculate AUC for every assay and then take the mean. For NN, ???
- Filtering NaN Values: Molecules with missing values (NaN) will be removed from the training pool. The neural network will only be trained with valid data, and only molecules with labels (0 or 1) will be considered in the test set.

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
from itertools import product

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

from rdkit import Chem


#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

print(torch.cuda.is_available())
print(torch.version.cuda)  # Check PyTorch's CUDA version
print(torch.cuda.device_count())  # Number of GPUs

True
12.4
1


In [2]:
# Load from pickle file
with open("datasets/datasets.pkl", "rb") as f:
    datasets = pd.read_pickle(f)

# Extract individual DataFrames
train_set_import = datasets["train"]
validation_set_import = datasets["validation"]
test_set_import = datasets["test"]

# Convert SMILES strings back to RDKit molecule objects
def postprocess_from_pickle(df):
    df = df.copy()
    df['molecule'] = df['molecule'].apply(lambda smiles: Chem.MolFromSmiles(smiles) if smiles else None)
    return df

# Apply postprocessing
train_set = postprocess_from_pickle(train_set_import)
validation_set = postprocess_from_pickle(validation_set_import)
test_set = postprocess_from_pickle(test_set_import)

[03:11:09] WARNING: not removing hydrogen atom without neighbors
[03:11:11] WARNING: not removing hydrogen atom without neighbors
[03:11:12] WARNING: not removing hydrogen atom without neighbors


In [3]:
train_set.head()

,molecule,quantilesXecfps,ATG_AR_TRANS_dn,TOX21_p53_BLA_p1_ratio,ATG_FoxA2_CIS_up,BSK_hDFCGF_TIMP1_down,ATG_LXRb_TRANS_up,Tanguay_ZF_120hpf_JAW_up,BSK_LPS_IL8_up,NVS_ADME_hCYP2D6,...,TOX21_ERa_BLA_Antagonist_ch1,NVS_ENZ_oCOX2,TOX21_ARE_BLA_Agonist_ch2,BSK_LPS_CD40_down,TOX21_Aromatase_Inhibition,BSK_hDFCGF_MCSF_down,ATG_RARa_TRANS_up,BSK_LPS_IL1a_up,TOX21_ESRE_BLA_ratio,NVS_ENZ_hBACE
0,<rdkit.Chem.rdchem.Mol object at 0x000001C6AEA...,"[1.3356484982591836, 1.3356484982591836, -0.38...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,<rdkit.Chem.rdchem.Mol object at 0x000001C6AEA...,"[0.9840356627736608, 0.9840356627736608, 0.100...",NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN
2,<rdkit.Chem.rdchem.Mol object at 0x000001C6AEA...,"[1.4329130642502799, 1.4329130642502799, -0.37...",NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN
3,<rdkit.Chem.rdchem.Mol object at 0x000001C6AEA...,"[1.3424432672295052, 1.3424432672295052, -0.39...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,<rdkit.Chem.rdchem.Mol object at 0x000001C6AEA...,"[1.303283942956551, 1.303283942956551, -0.3673...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [5]:
def compute_dauprc_score(predictions, labels, target_ids):
    """
    Computes the ΔAUC-PR score for each target separately and the mean ΔAUC-PR score.
    """
    dauprcs = list()
    target_id_list = list()

    for target_idx in torch.unique(target_ids):
        rows = torch.where(target_ids == target_idx)
        preds = predictions[rows].detach()
        y = labels[rows].int()

        if torch.unique(y).shape[0] == 2:
            number_actives = y[y == 1].shape[0]
            number_inactives = y[y == 0].shape[0]
            number_total = number_actives + number_inactives

            random_clf_auprc = number_actives / number_total
            auprc = average_precision_score(
                y.numpy().flatten(), preds.numpy().flatten()
            )

            dauprc = auprc - random_clf_auprc
            dauprcs.append(dauprc)
            target_id_list.append(target_idx.item())
        else:
            dauprcs.append(np.nan)
            target_id_list.append(target_idx.item())

    return np.nanmean(dauprcs), dauprcs, target_id_list

### Training & Evaluation Method

In [10]:
def train_and_evaluate(train_function, train_data, seeds, **kwargs):
    seed_results = {}

    for seed in seeds:
        print(f"Evaluating with seed {seed}...")
        np.random.seed(seed)  # Set seed für Reproduzierbarkeit
        results, overall_mean_auc, overall_std_auc, overall_mean_dauprc, overall_std_dauprc = train_function(train_data, **kwargs)
        
        seed_results[seed] = {
            'mean_auc': overall_mean_auc,
            'std_auc': overall_std_auc,
            'mean_dauprc': overall_mean_dauprc,
            'std_dauprc': overall_std_dauprc,  # Neu: Standardabweichung ΔAUC-PR
            'results': results
        }

    # Aggregierte Ergebnisse über alle Seeds berechnen
    all_mean_aucs = [res['mean_auc'] for res in seed_results.values()]
    all_std_aucs = [res['std_auc'] for res in seed_results.values()]
    all_mean_dauprcs = [res['mean_dauprc'] for res in seed_results.values()]
    all_std_dauprcs = [res['std_dauprc'] for res in seed_results.values()]

    overall_mean_auc = np.mean(all_mean_aucs)
    overall_std_auc = np.mean(all_std_aucs)
    overall_mean_dauprc = np.mean(all_mean_dauprcs)
    overall_std_dauprc = np.mean(all_std_dauprcs)  # Neu: Gesamt-Std für ΔAUC-PR

    # Erstelle eine Zusammenfassung als DataFrame
    summary_df = pd.DataFrame(
        data={'Training Method': ['Random Forest'], 
              'Mean AUC': [f"{overall_mean_auc:.4f}"],
              'Std AUC': [f"{overall_std_auc:.4f}"],
              'Mean ΔAUC-PR': [f"{overall_mean_dauprc:.4f}"],
              'Std ΔAUC-PR': [f"{overall_std_dauprc:.4f}"]})  # Neu: Std ΔAUC-PR

    # Drucken der Zusammenfassung
    print("\nSummary of Results:")
    print(summary_df.to_string(index=False))

    return seed_results, summary_df


### Random Forest

In [13]:
def random_forest(train_data, n_episodes=5):
    assay_columns = [col for col in train_data.columns 
                     if col not in ['molecule', 'quantilesXecfps']]
    
    assay_results = {}
    total_assays = len(assay_columns)

    with tqdm(total=total_assays, desc="Training Assays") as pbar:
        for assay in assay_columns:
            pbar.set_postfix(current_assay=assay)

            # Entferne NaN-Werte im aktuellen Assay
            assay_data = train_data.dropna(subset=[assay])
            positive_samples = assay_data[assay_data[assay] == 1]
            negative_samples = assay_data[assay_data[assay] == 0]

            if len(positive_samples) < 5 or len(negative_samples) < 5:
                pbar.update(1)
                continue

            episode_aucs = []
            episode_dauprcs = []  # Hier sammeln wir alle ΔAUC-PR Werte

            for _ in range(n_episodes):
                # Sample Support-Set
                support_pos = positive_samples.sample(n=5)
                support_neg = negative_samples.sample(n=5)
                support_set = pd.concat([support_pos, support_neg])

                # Erstelle das Query-Set (Rest der Daten)
                query_set = assay_data.drop(support_set.index)

                # Features und Labels vorbereiten
                X_support = np.stack(support_set['quantilesXecfps'].values)
                y_support = support_set[assay].values
                X_query = np.stack(query_set['quantilesXecfps'].values)
                y_query = query_set[assay].values

                # Trainiere den Random Forest
                rf = RandomForestClassifier(random_state=None) # Place seed here
                rf.fit(X_support, y_support)

                # Evaluieren auf Query-Set
                y_pred = rf.predict_proba(X_query)[:, 1]

                # Berechnung von ROC-AUC
                episode_auc = roc_auc_score(y_query, y_pred)
                episode_aucs.append(episode_auc)

                # Berechnung von ΔAUC-PR
                y_tensor = torch.tensor(y_query)
                preds_tensor = torch.tensor(y_pred)
                target_ids = torch.full_like(y_tensor, fill_value=assay_columns.index(assay), dtype=torch.int64)
                _, dauprc, _ = compute_dauprc_score(preds_tensor, y_tensor, target_ids)
                episode_dauprcs.append(dauprc)

            # Berechne den Mittelwert und die Standardabweichung für AUC und ΔAUC-PR
            mean_assay_auc = np.mean(episode_aucs)
            std_assay_auc = np.std(episode_aucs)
            mean_assay_dauprc = np.mean(episode_dauprcs)
            std_assay_dauprc = np.std(episode_dauprcs)

            assay_results[assay] = {
                'mean_auc': mean_assay_auc,
                'std_auc': std_assay_auc,
                'mean_dauprc': mean_assay_dauprc,
                'std_dauprc': std_assay_dauprc,  # Neu: Standardabweichung ΔAUC-PR
                'episode_aucs': episode_aucs,
                'episode_dauprcs': episode_dauprcs,  
            }

            pbar.update(1)

    # Gesamtwerte über alle Assays berechnen
    overall_mean_auc = np.mean([res['mean_auc'] for res in assay_results.values()])
    overall_std_auc = np.std([res['mean_auc'] for res in assay_results.values()])
    overall_mean_dauprc = np.mean([res['mean_dauprc'] for res in assay_results.values()])
    overall_std_dauprc = np.std([res['mean_dauprc'] for res in assay_results.values()])

    print(f"Overall Mean AUC across assays: {overall_mean_auc:.4f} ± {overall_std_auc:.4f}")
    print(f"Overall Mean ΔAUC-PR across assays: {overall_mean_dauprc:.4f} ± {overall_std_dauprc:.4f}")
    print("=====================================================================")
    print("")

    return assay_results, overall_mean_auc, overall_std_auc, overall_mean_dauprc, overall_std_dauprc


In [12]:
seeds = [0, 11942094, 23884188, 35826282, 47768376]
seed_results, summary_table = train_and_evaluate(
    train_function=random_forest,  # Pass your training function
    train_data=validation_set,  # Pass your dataset
    seeds=seeds,
    n_episodes=5  # Additional arguments for the training function
)


Evaluating with seed 0...


Training Assays:  93%|█████████▎| 114/123 [02:00<00:08,  1.05it/s, current_assay=ATG_Sp1_CIS_dn]                   

### Neural Network

In [8]:
# Set fixed seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Define the Frequent Hitters model with customizable hyperparameters
class FrequentHittersModel(nn.Module):
    def __init__(self, input_dim, hidden_layers=1, hidden_units=1024, dropout=0.4, layer_norm=False):
        super(FrequentHittersModel, self).__init__()
        layers = []
        for _ in range(hidden_layers):
            layers.append(nn.Linear(input_dim, hidden_units))
            if layer_norm:
                layers.append(nn.LayerNorm(hidden_units))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            input_dim = hidden_units
        layers.append(nn.Linear(input_dim, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.model(x))

# Function to train Frequent Hitters Model and compute AUC & ΔAUC-PR
def train_frequent_hitters(train_data, hyperparam_grid, seed=42):
    set_seed(seed)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    assay_columns = [col for col in train_data.columns if col not in ['molecule', 'quantilesXecfps']]
    assay_results = {}
    best_model_params = None
    best_overall_auc = 0

    with tqdm(total=len(assay_columns), desc="Training Assays") as pbar:
        for assay in assay_columns:
            train_assay_data = train_data.dropna(subset=[assay])
            positive_samples = train_assay_data[train_assay_data[assay] == 1]
            negative_samples = train_assay_data[train_assay_data[assay] == 0]

            if len(positive_samples) < 5 or len(negative_samples) < 5:
                pbar.update(1)
                continue

            best_assay_auc = 0
            best_assay_params = None
            episode_aucs = []
            episode_dauprcs = []

            for hyperparams in product(*hyperparam_grid.values()):
                hyperparams_dict = dict(zip(hyperparam_grid.keys(), hyperparams))

                for _ in range(5):  # 5 episodes
                    support_pos = positive_samples.sample(n=5, random_state=random.randint(0, 10000))
                    support_neg = negative_samples.sample(n=5, random_state=random.randint(0, 10000))
                    support_set = pd.concat([support_pos, support_neg])
                    query_set = train_assay_data.drop(support_set.index)

                    X_support = torch.tensor(np.stack(support_set['quantilesXecfps'].values), dtype=torch.float32).to(device)
                    y_support = torch.tensor(support_set[assay].values, dtype=torch.float32).to(device)
                    X_query = torch.tensor(np.stack(query_set['quantilesXecfps'].values), dtype=torch.float32).to(device)
                    y_query = torch.tensor(query_set[assay].values, dtype=torch.float32).to(device)

                    model = FrequentHittersModel(
                        input_dim=X_query.shape[1],
                        hidden_layers=hyperparams_dict['hidden_layers'],
                        hidden_units=hyperparams_dict['hidden_units'],
                        dropout=hyperparams_dict['dropout'],
                        layer_norm=hyperparams_dict['layer_norm']
                    ).to(device)

                    criterion = nn.BCELoss()
                    optimizer = optim.Adam(model.parameters(), lr=hyperparams_dict['learning_rate'], weight_decay=hyperparams_dict['weight_decay'])

                    model.train()
                    for epoch in range(hyperparams_dict['epochs']):
                        optimizer.zero_grad()
                        outputs = model(X_query)
                        loss = criterion(outputs, y_query.unsqueeze(1))
                        loss.backward()
                        optimizer.step()

                    # Evaluate on support set
                    model.eval()
                    with torch.no_grad():
                        y_pred = model(X_support).cpu().numpy().squeeze()

                    # Compute AUC
                    episode_auc = roc_auc_score(y_support.cpu(), y_pred)
                    episode_aucs.append(episode_auc)

                    # Compute ΔAUC-PR
                    target_ids = torch.zeros_like(y_support.cpu(), dtype=torch.int64) # Fix this
                    _, dauprc, _ = compute_dauprc_score(torch.tensor(y_pred), y_support.cpu(), target_ids)
                    episode_dauprcs.append(dauprc)

                # Compute mean and std for AUC and ΔAUC-PR
                mean_assay_auc = np.mean(episode_aucs)
                std_assay_auc = np.std(episode_aucs)
                mean_assay_dauprc = np.nanmean(episode_dauprcs)  # Avoid NaN issues
                std_assay_dauprc = np.nanstd(episode_dauprcs)

                if mean_assay_auc > best_assay_auc:
                    best_assay_auc = mean_assay_auc
                    best_assay_params = hyperparams_dict

                    if mean_assay_auc > best_overall_auc:
                        best_overall_auc = mean_assay_auc
                        best_model_params = hyperparams_dict

            assay_results[assay] = {
                'best_params': best_assay_params,
                'best_auc': best_assay_auc,
                'mean_dauprc': mean_assay_dauprc,
                'std_dauprc': std_assay_dauprc
            }

            pbar.update(1)

    # Compute overall metrics
    all_mean_aucs = [res['best_auc'] for res in assay_results.values()]
    all_std_aucs = [np.std(res['best_auc']) for res in assay_results.values()]
    all_mean_dauprcs = [res['mean_dauprc'] for res in assay_results.values()]
    all_std_dauprcs = [res['std_dauprc'] for res in assay_results.values()]

    overall_mean_auc = np.mean(all_mean_aucs)
    overall_std_auc = np.std(all_std_aucs)
    overall_mean_dauprc = np.nanmean(all_mean_dauprcs)
    overall_std_dauprc = np.nanstd(all_std_dauprcs)

    print(f"Overall Mean AUC: {overall_mean_auc:.4f} ± {overall_std_auc:.4f}")
    print(f"Overall Mean ΔAUC-PR: {overall_mean_dauprc:.4f} ± {overall_std_dauprc:.4f}")

    return assay_results, best_model_params, overall_mean_auc, overall_std_auc, overall_mean_dauprc, overall_std_dauprc


# Example usage
hyperparam_grid = {
    'hidden_layers': [1],
    'hidden_units': [1024],
    'dropout': [0.5],
    'layer_norm': [False],
    'learning_rate': [0.001],
    'weight_decay': [0],
    'batch_size': [128],
    'epochs': [5]
}

results, best_model_params, overall_mean_auc, overall_std_auc, overall_mean_dauprc, overall_std_dauprc = train_frequent_hitters(validation_set, hyperparam_grid, seed=12440)

Using device: cuda


Training Assays: 100%|██████████| 123/123 [01:01<00:00,  2.00it/s]

Overall Mean AUC: 0.5553 ± 0.0000
Overall Mean ΔAUC-PR: 0.1498 ± 0.0374


In [6]:
best_model_params

{'hidden_layers': 1,
 'hidden_units': 1024,
 'dropout': 0.5,
 'layer_norm': False,
 'learning_rate': 0.001,
 'weight_decay': 0,
 'batch_size': 128,
 'epochs': 5}

### Hyperparameter Search

### Seed Testing

In [ ]:
hyperparam_grid = {
    'hidden_layers': [1, 2, 4],
    'hidden_units': [1024, 2048],
    'dropout': [0.4, 0.6],
    'layer_norm': [False, True],
    'learning_rate': [0.0001, 0.001],
    'weight_decay': [0, 0.01],
    'batch_size': [128, 512],
    'epochs': [30]
}
hyperparam_grid = {
    'hidden_layers': [1],
    'hidden_units': [1024],
    'dropout': [0.1],
    'layer_norm': [False],
    'learning_rate': [0.001],
    'weight_decay': [0],
    'batch_size': [128],
    'epochs': [30]
}
seeds = [0, 11942094, 23884188, 35826282, 47768376]
seeds = [0]
results = train_all_assays(train_set, validation_set, test_set, hyperparam_grid, seeds)

In [6]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

0


RuntimeError: No CUDA GPUs are available

### Visualization